In [2]:
import spacy

In [3]:
#!python -m spacy download en_core_web_md

In [4]:
nlp = spacy.load('en_core_web_md')

In [5]:
import wikipedia

text = wikipedia.summary("United States")

In [6]:
doc = nlp(text)

In [14]:
sentence1 = list(doc.sents)[0]
print (sentence1)

The United States of America (USA or U.S.A.), commonly known as the United States (US or U.S.) or America, is a country primarily located in North America.


In [15]:
import numpy as np

In [16]:
word_to_search = 'Country'

In [17]:
ms = nlp.vocab.vectors.most_similar(
    np.asarray([nlp.vocab.vectors[nlp.vocab.strings[word_to_search]]]), n=10)
words = [nlp.vocab.strings[w] for w in ms[0][0]]
distance = ms[2]
print(words)

['Intercountry', 'Strathtay', 'Southerns', 'Heathlands', 'Facies', 'Communitas', 'Betties', 'Norteños', 'Trencsén', 'Regionalverkehr']


In [18]:
doc1= nlp('I enjoy apples')
doc2= nlp('I enjoy climbing trees')
print(doc1, '<-->', doc2, doc1.similarity(doc2))

I enjoy apples <--> I enjoy climbing trees 0.9021378059432545


# Entity Ruler

In [19]:
nlp = spacy.load('en_core_web_sm')

In [22]:
from spacy.vocab import Vocab
vocab = Vocab(strings=["hello", "world"])
vocab

In [23]:
from spacy.tokens import Doc
from spacy.training import Example

predicted = Doc(vocab, words=["Apply", "some", "sunscreen"])
token_ref = ["Apply", "some", "sun", "screen"]
tags_ref = ["VERB", "DET", "NOUN", "NOUN"]
example = Example.from_dict(predicted, {"words": token_ref, "tags": tags_ref})

In [24]:
example

{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O'], 'spans': {}, 'links': {}}, 'token_annotation': {'ORTH': ['Apply', 'some', 'sun', 'screen'], 'SPACY': [True, True, True, True], 'TAG': ['VERB', 'DET', 'NOUN', 'NOUN'], 'LEMMA': ['', '', '', ''], 'POS': ['', '', '', ''], 'MORPH': ['', '', '', ''], 'HEAD': [0, 1, 2, 3], 'DEP': ['', '', '', ''], 'SENT_START': [1, 0, 0, 0]}}